# Entities > Media Images Only
Extraction, Download & Visualization

###Load Libraries

In [1]:
import json
import numpy as np
import pandas as pd
import requests
from urllib.parse import urlparse
import shutil
import csv
import resource
#import pygal

###Filepaths

In [2]:
filename = sys.argv[1] #'/big_data/nepal/test.json'
# image_directory = '/Users/cbopp/Data/Nepal/images/' #Include trailing slash
output_file = sys.argv[2] #'/big_data/nepal/nepal_9-17-15.tsv'

###Limit Memory Usage

In [3]:
rsrc = resource.RLIMIT_DATA
soft, hard = resource.getrlimit(rsrc)
resource.setrlimit(rsrc, (2000000000, hard)) #2GB
soft, hard = resource.getrlimit(rsrc)

###Import JSON to Data Frame

In [4]:
l = []

print('Opening file...', end="", flush=True)
# count = 0
for line in open(filename):
    l.append(json.loads(line))
#     count+=1
#     if(count%100==0):
#         print(count)
print('File loaded.', flush=True)

print('Converting to DataFrame and doing some housekeeping...', end="", flush=True)
df = pd.DataFrame(l)
df['created_at'] = pd.to_datetime(df['created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
df['tweet_created_at_epoch'] = df['created_at'].astype('int64')
df['tweet_text'] = df['text'].apply(lambda s: s.replace('\t', '').replace('\r', '').replace('\n', ''))
df['tweet_in_reply_to_user_id_str'] = df['in_reply_to_user_id_str'].apply(lambda n: str(n))
df['tweet_in_reply_to_user_id_str'].replace('None', np.NaN, inplace=True)
df['tweet_source'] = df['source']
df['tweet_coords'] = df['coordinates']
df['tweet_language'] = df['lang']
print('Done.', flush=True)

Opening file...File loaded.
Converting to DataFrame and doing some housekeeping...Done.


###Identify Tweets that Contain Entities

In [5]:
e_media_data = {'tweet_id_str': [], 
                'media_url': [], 
                'image_id_str': []}

print('Extracting media URLs...', end="", flush=True)
for tweet in l:
    for entity in tweet['entities']:
        if entity == 'media':
            for image in tweet['entities'][entity]:
                e_media_data['tweet_id_str'].append(tweet['id_str'])
                e_media_data['media_url'].append(image['media_url'])
                e_media_data['image_id_str'].append(image['id_str'])

e_media = pd.DataFrame(e_media_data, columns=['tweet_id_str', 'image_id_str', 'media_url'])
print('Done.', flush=True)

Extracting media URLs...Done.


###Get Images

In [6]:
#print('Preparing and merging DataFrames...', end="", flush=True)
#e_media['saved_to_filename'] = e_media.apply(lambda e: (e.tweet_id_str + "-" + e.media_url[7:]).replace("/","_"), axis=1)

In [7]:
tweets_with_e_media = pd.merge(e_media, df, left_on='tweet_id_str', right_on='id_str', how='left')
print('Done.', flush=True)

Done.


In [8]:
#TODO: Deal with images that are gzip'ed (even though the extension is a jpg for example)
#def download_images(row):

    #url = "http://pbs.twimg.com/media" + urlparse(row['media_url']).path #Local
    ##url = "http://pbs.twimg.com" + urlparse(row['media_url']).path #Server

    #response = requests.get(url, stream=True)
    #if response.status_code == 200:
    #    with open(image_directory + row['saved_to_filename'], 'wb') as out_file:
    #        shutil.copyfileobj(response.raw, out_file)
    #    return "1"
    #else:
    #    return "0"

#print('Downloading images...', end="", flush=True)
#tweets_with_e_media['download_status'] = tweets_with_e_media.apply(download_images, axis=1)
#print('Done.', flush=True)

###Merge in Other Relevant Info

In [2]:
user_data = {'tweet_id_str': [],
             'user_created_at': [],
             'user_id_str': [],
             'user_default_avatar': [],
             'user_followers_count': [],
             'user_following_count': [],
             'user_listed_count': [], 
             'user_name': [],
             'user_screen_name': [], 
             'user_tweet_count': [], 
             'user_language': [], 
             'user_location': [],
             'user_geo_enabled': [],
             'user_verified': []}

print('Extracting retweet user info...', end="", flush=True)

for tweet in l:
    user_data['tweet_id_str'].append(tweet['id_str'])
    user_data['user_created_at'].append(tweet['user']['created_at'])
    user_data['user_id_str'].append(tweet['user']['id_str'])
    user_data['user_default_avatar'].append(tweet['user']['default_profile_image'])
    user_data['user_followers_count'].append(tweet['user']['followers_count'])
    user_data['user_following_count'].append(tweet['user']['friends_count'])
    user_data['user_listed_count'].append(tweet['user']['listed_count'])
    user_data['user_name'].append(tweet['user']['name'])
    user_data['user_screen_name'].append(tweet['user']['screen_name'])
    user_data['user_tweet_count'].append(tweet['user']['statuses_count'])
    user_data['user_language'].append(tweet['user']['lang'])
    user_data['user_location'].append(tweet['user']['location'])
    user_data['user_geo_enabled'].append(tweet['user']['geo_enabled'])
    user_data['user_verified'].append(tweet['user']['verified'])
        
user = pd.DataFrame(user_data, columns=[
        'tweet_id_str', 
        'user_created_at',
        'user_id_str',
        'user_default_avatar',
        'user_followers_count',
        'user_following_count',
        'user_listed_count',
        'user_name',
        'user_screen_name',
        'user_tweet_count',
        'user_language',
        'user_location',
        'user_geo_enabled',
        'user_verified'])

user['user_created_at'] = pd.to_datetime(user['user_created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
user['user_created_at_epoch'] = user['user_created_at'].astype('int64')
user['user_name'] = user['user_name'].apply(lambda s: s.replace('\t', '').replace('\r', '').replace('\n', ''))

print('Done.', flush=True)

SyntaxError: invalid syntax (<ipython-input-2-8029d875cc93>, line 11)

In [1]:
retweeted_status_data = {'tweet_id_str': [],
                         'rt': [],
                         'original_tweet_created_at': [],
                         'original_tweet_id_str': [], 
                         'original_user_created_at': [],
                         'original_user_id_str': [],
                         'original_user_default_avatar': [],
                         'original_user_followers_count': [],
                         'original_user_following_count': [],
                         'original_user_listed_count': [],
                         'original_user_name': [],
                         'original_user_screen_name': [],
                         'original_user_tweet_count': [],
                         'original_user_verified': [],
                         'original_user_language': [],
                         'original_user_location': [],
                         'original_user_geo_enabled': [],
                         'original_tweet_coords': [],                         
                         'original_tweet_language': [],                         
                         'original_tweet_text': [],                         
                         'original_tweet_source': []}

print('Extracting retweet status...', end="", flush=True)

for tweet in l:  
    try:
        tweet['retweeted_status']
        rt = True
    except:
        rt = False
        
    if rt:
        retweeted_status_data['tweet_id_str'].append(tweet['id_str'])
        retweeted_status_data['rt'].append(rt)
        retweeted_status_data['original_tweet_created_at'].append(tweet['retweeted_status']['created_at'])
        retweeted_status_data['original_tweet_id_str'].append(tweet['retweeted_status']['id_str'])
        retweeted_status_data['original_user_created_at'].append(tweet['retweeted_status']['user']['created_at'])
        retweeted_status_data['original_user_id_str'].append(tweet['retweeted_status']['user']['id_str'])
        retweeted_status_data['original_user_default_avatar'].append(tweet['retweeted_status']['user']['default_profile_image'])
        retweeted_status_data['original_user_followers_count'].append(tweet['retweeted_status']['user']['followers_count'])
        retweeted_status_data['original_user_following_count'].append(tweet['retweeted_status']['user']['friends_count'])
        retweeted_status_data['original_user_listed_count'].append(tweet['retweeted_status']['user']['listed_count'])
        retweeted_status_data['original_user_name'].append(tweet['retweeted_status']['user']['name'])
        retweeted_status_data['original_user_screen_name'].append(tweet['retweeted_status']['user']['screen_name'])
        retweeted_status_data['original_user_tweet_count'].append(tweet['retweeted_status']['user']['statuses_count'])
        retweeted_status_data['original_user_verified'].append(tweet['retweeted_status']['user']['verified'])
        retweeted_status_data['original_user_language'].append(tweet['retweeted_status']['user']['lang'])
        retweeted_status_data['original_user_location'].append(tweet['retweeted_status']['user']['location'])
        retweeted_status_data['original_user_geo_enabled'].append(tweet['retweeted_status']['user']['geo_enabled'])
        retweeted_status_data['original_tweet_coords'].append(tweet['retweeted_status']['coordinates'])
        retweeted_status_data['original_tweet_language'].append(tweet['retweeted_status']['lang'])
        retweeted_status_data['original_tweet_text'].append(tweet['retweeted_status']['text'])
        retweeted_status_data['original_tweet_source'].append(tweet['retweeted_status']['source'])
    else:
        retweeted_status_data['tweet_id_str'].append(tweet['id_str'])
        retweeted_status_data['rt'].append(rt)
        retweeted_status_data['original_tweet_created_at'].append(np.NaN)
        retweeted_status_data['original_tweet_id_str'].append(np.NaN)
        retweeted_status_data['original_user_created_at'].append(np.NaN)
        retweeted_status_data['original_user_id_str'].append(np.NaN)
        retweeted_status_data['original_user_default_avatar'].append(np.NaN)
        retweeted_status_data['original_user_followers_count'].append(np.NaN)
        retweeted_status_data['original_user_following_count'].append(np.NaN)
        retweeted_status_data['original_user_listed_count'].append(np.NaN)
        retweeted_status_data['original_user_name'].append(np.NaN)
        retweeted_status_data['original_user_screen_name'].append(np.NaN)
        retweeted_status_data['original_user_tweet_count'].append(np.NaN)
        retweeted_status_data['original_user_verified'].append(np.NaN)
        retweeted_status_data['original_user_language'].append(np.NaN)
        retweeted_status_data['original_user_location'].append(np.NaN)
        retweeted_status_data['original_user_geo_enabled'].append(np.NaN)
        retweeted_status_data['original_tweet_coords'].append(np.NaN)
        retweeted_status_data['original_tweet_language'].append(np.NaN)
        retweeted_status_data['original_tweet_text'].append(np.NaN)
        retweeted_status_data['original_tweet_source'].append(np.NaN)

retweeted_status = pd.DataFrame(retweeted_status_data, columns=[
        'tweet_id_str',
        'rt',
        'original_tweet_created_at', 
        'original_tweet_id_str',
        'original_user_created_at',
        'original_user_id_str',
        'original_user_default_avatar',
        'original_user_followers_count',
        'original_user_following_count',
        'original_user_listed_count',
        'original_user_name',
        'original_user_screen_name',
        'original_user_tweet_count',
        'original_user_verified',
        'original_user_language',
        'original_user_location',
        'original_user_geo_enabled',
        'original_tweet_coords',
        'original_tweet_language',
        'original_tweet_text',
        'original_tweet_source'])

retweeted_status['original_tweet_created_at'] = pd.to_datetime(retweeted_status['original_tweet_created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
retweeted_status['original_user_created_at'] = pd.to_datetime(retweeted_status['original_user_created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
retweeted_status['original_tweet_created_at_epoch'] = retweeted_status['original_tweet_created_at'].astype('int64')
retweeted_status['original_user_created_at_epoch'] = retweeted_status['original_user_created_at'].astype('int64')
retweeted_status['original_user_name'] = retweeted_status['original_user_name'].apply(lambda s: str(s).replace('\t', '').replace('\r', '').replace('\n', ''))
retweeted_status['original_tweet_text'] = retweeted_status['original_tweet_text'].apply(lambda s: str(s).replace('\t', '').replace('\r', '').replace('\n', ''))

print('Done.', flush=True)

SyntaxError: invalid syntax (<ipython-input-1-c340a2ce364a>, line 6)

In [11]:
print('Merging DataFrames...', end="", flush=True)
tweets_with_e_media = pd.merge(tweets_with_e_media, user, left_on='tweet_id_str', right_on='tweet_id_str', how='left')
tweets_with_e_media = pd.merge(tweets_with_e_media, retweeted_status, on='tweet_id_str', how='left')
print('Done.', flush=True)

Merging DataFrames...Done.


###Binning Data/Reformatting for Output

In [12]:
#source_replacements = {
#    'web':'0',
#    '<a href="http://mobile.twitter.com" rel="nofollow">Mobile Web</a>':'1',
#    '<a href="http://twitter.com/devices" rel="nofollow">txt</a>':'2',
#    '<a href="http://twitter.com/tweetbutton" rel="nofollow">Tweet Button</a>':'3',
#    '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>':'4',
#    '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>':'5',
#    '<a href="http://tapbots.com/tweetbot" rel="nofollow">Tweetbot for iOS</a>':'6',
#    '<a href="http://www.apple.com" rel="nofollow">iOS</a>':'7',
#    '<a href="http://itunes.apple.com/us/app/ap-mobile/id284901416?mt=8&uo=4" rel="nofollow">AP Mobile on iOS</a>':'8',
#    '<a href="http://itunes.apple.com/us/app/twitter/id409789998?mt=12" rel="nofollow">Twitter for Mac</a>':'9',
#    '<a href="http://tapbots.com/software/tweetbot/mac" rel="nofollow">Tweetbot for Mac</a>':'10',
#    '<a href="http://www.apple.com" rel="nofollow">Safari on iOS</a>':'11',
#    '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>':'12',
#    '<a href="http://www.tweetcaster.com" rel="nofollow">TweetCaster for Android</a>':'13',
#    '<a href="http://levelupstudio.com/plume" rel="nofollow">Plume for Android</a>':'14',
#    '<a href="http://blackberry.com/twitter" rel="nofollow">Twitter for BlackBerry®</a>':'15',
#    '<a href="http://www.tweetdeck.com" rel="nofollow">TweetDeck</a>':'16',
#    '<a href="http://www.hootsuite.com" rel="nofollow">HootSuite</a>':'17',
#    '<a href="http://www.google.com/" rel="nofollow">Google</a>':'18',
#    '<a href="http://twitterfeed.com" rel="nofollow">twitterfeed</a>':'19',
#    '<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>':'20',
#    '<a href="http://www.echofon.com/" rel="nofollow">Echofon</a>':'21',
#    '<a href="http://www.socialnewsdesk.com" rel="nofollow">SocialNewsDesk</a>':'22',
#    '<a href="http://dlvr.it" rel="nofollow">dlvr.it</a>':'23',
#    '<a href="http://twitterrific.com" rel="nofollow">Twitterrific</a>':'24'
#}

#tweets_with_e_media.replace({'source': source_replacements}, inplace=True)

In [13]:
tweets_with_e_media['rt'].replace(True, '1', inplace=True)
tweets_with_e_media['rt'].replace(False, '0', inplace=True)
tweets_with_e_media['user_default_avatar'].replace(True, '1', inplace=True)
tweets_with_e_media['user_default_avatar'].replace(False, '0', inplace=True)
tweets_with_e_media['user_verified'].replace(True, '1', inplace=True)
tweets_with_e_media['user_verified'].replace(False, '0', inplace=True)
tweets_with_e_media['user_geo_enabled'].replace(True, '1', inplace=True)
tweets_with_e_media['user_geo_enabled'].replace(False, '0', inplace=True)
tweets_with_e_media['original_user_default_avatar'].replace(True, '1', inplace=True)
tweets_with_e_media['original_user_default_avatar'].replace(False, '0', inplace=True)
tweets_with_e_media['original_user_verified'].replace(True, '1', inplace=True)
tweets_with_e_media['original_user_verified'].replace(False, '0', inplace=True)
tweets_with_e_media['original_user_geo_enabled'].replace(True, '1', inplace=True)
tweets_with_e_media['original_user_geo_enabled'].replace(False, '0', inplace=True)

###Output to Visualization

In [14]:
#TODO: Remove 000000000 (trailing zeros) from all epoch times

In [15]:
print('Writing data file...', end="", flush=True)
#tweets_with_e_media.to_csv(output_file, sep='\t', na_rep='Null', columns=[
#    'saved_to_filename', 'download_status', 'media_url', 'image_id_str', 'retweet_id_str', 'retweet_created_at_epoch', 'retweet_source', 
#    'retweet_user_id_str', 'retweet_user_default_avatar','retweet_user_followers_count', 'retweet_user_following_count', 'retweet_user_listed_count', 
#    'retweet_user_name', 'retweet_user_screen_name', 'retweet_user_tweet_count', 'retweet_user_verified', 'retweet_user_created_at_epoch',
#    'retweet_in_reply_to_user_id_str', 'rt', 
#    'original_tweet_id_str', 'original_tweet_created_at_epoch', 'original_source', 'original_user_id_str', 'original_user_created_at_epoch', 
#    'original_user_default_avatar', 'original_user_followers_count', 'original_user_following_count', 
#    'original_user_listed_count', 'original_user_name', 'original_user_screen_name', 'original_user_tweet_count', 'original_user_verified', 
#    'retweet_text', 'original_text'], index=False)
tweets_with_e_media.to_csv(output_file, sep='\t', na_rep='Null', columns=[
        'media_url', 
        'image_id_str',
        'tweet_id_str',
        'tweet_created_at_epoch',
        'tweet_source',
        'tweet_coords',
        'tweet_language',
        'user_id_str',
        'user_default_avatar',
        'user_followers_count',
        'user_following_count',
        'user_listed_count',
        'user_name',
        'user_screen_name',
        'user_tweet_count',
        'user_language',
        'user_location',
        'user_geo_enabled',
        'user_verified',
        'user_created_at_epoch',
        'tweet_in_reply_to_user_id_str',
        'rt',
        'original_tweet_id_str',
        'original_tweet_created_at_epoch',
        'original_tweet_source',
        'original_user_id_str',
        'original_user_created_at_epoch',
        'original_user_default_avatar',
        'original_user_followers_count',
        'original_user_following_count',
        'original_user_listed_count',
        'original_user_name',
        'original_user_screen_name',
        'original_user_tweet_count',
        'original_user_verified',
        'original_user_language',
        'original_user_location',
        'original_user_geo_enabled',
        'original_tweet_coords',
        'original_tweet_language',
        'tweet_text',
        'original_tweet_text'], index=False)
print('Done.', flush=True)

Writing data file...Done.
